In [ ]:
from fastai.vision.all import *
import fastai
import timm
import albumentations as A

# In[2]:


import torch
torch.cuda.set_device(0)

In [ ]:
dfNew = pd.read_csv('multi-label-annotation.csv')

In [ ]:
dfNew=dfNew[dfNew.tags!='NRG']

In [ ]:
dfNew

In [ ]:
def is_valid(path):
    name = path[0]
    return (dfNew[dfNew['Eye ID']==name])['label'].values[0]=='valid'


db = DataBlock(blocks = (ImageBlock, MultiCategoryBlock),
             splitter=FuncSplitter(is_valid),
             get_x = ColReader(-3),
             get_y=ColReader(-1,label_delim=' '),
             item_tfms = [Resize(384)], # CropPad(200,200)
             batch_tfms=[*aug_transforms(size=384, min_scale=0.75,do_flip=True,flip_vert=True,
                  max_rotate=2.,max_zoom=1.1, max_warp=0.05,p_affine=0.9, p_lighting=0.8), 
                         Normalize.from_stats(*imagenet_stats)])


In [ ]:
dls = db.dataloaders(dfNew.values,bs=8)

In [ ]:
dls.show_batch(max_n=9, figsize=(12,9))

In [ ]:
learn_multi_label = load_learner('convnext_base_384_in22ft1k_multi_labelv3.pkl')

In [ ]:
learn_multi_label.dls = dls

In [ ]:
preds,gt = learn_multi_label.get_preds()

In [ ]:
def hamming_loss(true_labels, predicted_labels):
    """Calculate the Hamming loss for the given true and predicted labels."""
    # Convert to numpy arrays for efficient computation
    true_labels = np.array(true_labels)
    predicted_labels = np.array(predicted_labels)

    # Calculate the hamming distance that is basically the total number of mismatches
    Hamming_distance = np.sum(np.not_equal(true_labels, predicted_labels))
    print("Hamming distance", Hamming_distance)
    
    # Calculate the total number of labels
    total_corrected_labels= true_labels.size

    # Compute the Modified Hamming loss
    loss = Hamming_distance / total_corrected_labels
    return loss

In [ ]:
preds

In [ ]:
hamming_loss(gt,preds > 0.5)

In [ ]:
learn_multi_label.remove_cbs(learn_multi_label.cbs)

In [ ]:
learn_multi_label.remove_cbs([Recorder,ProgressCallback,SaveModelCallback])

In [ ]:
predstta,_ = learn_multi_label.tta()

In [ ]:
hamming_loss(gt,predstta > 0.5)